In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
from SlideRunner.dataAccess.database import *
from pathlib import Path
import pandas as pd
import json
import sys

In [3]:
slides_path = Path("")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")}
len(files)

94

In [4]:
df = pd.read_pickle("../SDATA/Statistics/SDATA_Final_Annotations.pkl")
df.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,user_id,deleted,last_editor,data_set_name,version
unique_identifier,,,,,,,,,,,,,
58c6d630-9031-47be-b0f6-a74d8531041d,2643954,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 24446.0, 'x2': 24593.0, 'y1': 23891.0, ...",1,False,1,SDATA,Density
aadd3497-a7bb-4e91-82b1-b2fbfdc08d6d,2643955,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 15457.0, 'x2': 15578.0, 'y1': 24909.0, ...",1,False,1,SDATA,Density
f535a6f4-9311-43e8-8ff3-674f232dc515,2643956,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 20301.0, 'x2': 20410.0, 'y1': 13917.0, ...",1,False,1,SDATA,Density
c2853048-9036-41cd-8598-dcf2db65f834,2643957,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 26649.0, 'x2': 26791.0, 'y1': 13089.0, ...",1,False,1,SDATA,Density
1b1549b0-4a44-4f12-b167-fdc2f7429843,2643958,6910,284,Cat,BAL_Cat_Turnbull blue_14.svs,Turnbull,0,"{'x1': 14791.0, 'x2': 14941.0, 'y1': 11796.0, ...",1,False,1,SDATA,Density


In [5]:
DB = Database()

In [6]:
DB.create("SDATA_Final_Annotations.sqlite")

In [7]:
DB.insertAnnotator('EXACT')

1

In [8]:
DB.insertClass('0')
DB.insertClass('1')
DB.insertClass('2')
DB.insertClass('3')
DB.insertClass('4')

In [9]:
classes = DB.getAllClasses()
classes = {name:id for name, id in  classes}
classes

{'0': 1, '1': 2, '2': 3, '3': 4, '4': 5}

In [10]:
t = 0
for slide in tqdm(df["image_name"].unique()):
    
    imageDf = df[df["image_name"]==slide]
    
    image_path = str(files[slide])
    
    image_id = DB.insertNewSlide(slide,image_path)
    image_id = DB.findSlideWithFilename(slide,image_path)
    
    for label, vector in zip(imageDf['grade'], imageDf['vector']):
        
        try:
        
            label_id = classes[str(label)]
            vector = json.loads(vector) if type(vector) is str else  vector

            DB.insertNewAreaAnnotation(x1=int(vector['x1']),
                                       y1=int(vector['y1']),
                                       x2=int(vector['x2']),
                                       y2=int(vector['y2']), 
                                       slideUID=image_id, 
                                       classID=label_id, 
                                       annotator=1) 
        except :
            t += 1
    
    

100%|███████████████████████████████████████████████████████████████████████████████| 74/74 [3:43:27<00:00, 181.19s/it]
